# Classical Statistics Approach

Let's start with loading our data, splitting it, then training a model on all possible input variables.

In [2]:
import statsmodels.api as sm
from statsmodels.tools.eval_measures import aic
import pandas as pd
from sklearn.model_selection import train_test_split
import math

In [31]:
df1 = pd.read_csv("bankruptcy.csv")
df1['Intercept'] = 1  # allows the regression to have an intercept
x_train, x_test = train_test_split(df1[['X1', 'X2', 'X3', 'X4', 'Intercept']], 
                                   test_size=0.2, random_state=120)
y_train, y_test = train_test_split(df1['Group'], test_size=0.2, random_state=120)

In [30]:
full_model = sm.Logit(y_train, x_train)
full_model_fit = full_model.fit()
print(full_model_fit.summary())

Optimization terminated successfully.
         Current function value: 0.317232
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:                  Group   No. Observations:                   36
Model:                          Logit   Df Residuals:                       31
Method:                           MLE   Df Model:                            4
Date:                Tue, 26 Mar 2024   Pseudo R-squ.:                  0.5382
Time:                        19:23:38   Log-Likelihood:                -11.420
converged:                       True   LL-Null:                       -24.731
Covariance Type:            nonrobust   LLR p-value:                 2.372e-05
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
X1             6.2847      5.729      1.097      0.273      -4.945      17.514
X2            -3.6808     12.

Now we can see about narrowing the model down. Since the $X_2$ and $X_4$  $p$-values are so high, let's see how the model would fair without them, and compare the Akaike information criterion (AIC) values for the reduced and full models. First, let's just take out $X_1$ for our reduced model, and see how it compares.

In [43]:
# fitting model without X_2
x_train2 = x_train[['X1', 'X3', 'X4', 'Intercept']]
reduced_model1 = sm.Logit(y_train, x_train2)
reduced_model1_fit = reduced_model1.fit()

full_model_aic = aic(llf=full_model.loglike(full_model_fit.params), 
                     nobs=x_train.size, 
                     df_modelwc=len(full_model_fit.params))
print("\n   Full model AIC: {:.3f}".format(full_model_aic))

reduced_model1_aic = aic(llf=reduced_model1.loglike(reduced_model1_fit.params), 
                        nobs=x_train2.shape[0], 
                        df_modelwc=len(reduced_model1_fit.params))
print("Reduced model AIC: {:.3f}".format(reduced_model1_aic))

Optimization terminated successfully.
         Current function value: 0.318429
         Iterations 8

   Full model AIC: 32.841
Reduced model AIC: 30.927


The AIC of a model measures how much information from the original data is <i>lost</i> by the model, so a lower AIC is better. This means the reduced model is better in comparison to the full model. To see if removing another variable improves it further, let's look at the summary of the reduced model:

In [45]:
print(reduced_model1_fit.summary())

                           Logit Regression Results                           
Dep. Variable:                  Group   No. Observations:                   36
Model:                          Logit   Df Residuals:                       32
Method:                           MLE   Df Model:                            3
Date:                Tue, 26 Mar 2024   Pseudo R-squ.:                  0.5365
Time:                        19:33:40   Log-Likelihood:                -11.463
converged:                       True   LL-Null:                       -24.731
Covariance Type:            nonrobust   LLR p-value:                 7.371e-06
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
X1             4.8697      2.909      1.674      0.094      -0.831      10.570
X3             2.8358      1.051      2.699      0.007       0.776       4.895
X4            -2.2554      3.764     -0.599      0.5

$X_4$ seems to be our next candidate to elimate. So let's try it and see how it goes:

In [46]:
# fitting model without X_2 and X_4
x_train3 = x_train[['X1', 'X3', 'Intercept']]
reduced_model2 = sm.Logit(y_train, x_train3)
reduced_model2_fit = reduced_model2.fit()

print("\n1st reduced model AIC: {:.3f}".format(reduced_model1_aic))

reduced_model2_aic = aic(llf=reduced_model2.loglike(reduced_model2_fit.params), 
                        nobs=x_train3.shape[0], 
                        df_modelwc=len(reduced_model2_fit.params))
print("2nd reduced model AIC: {:.3f}".format(reduced_model2_aic))

Optimization terminated successfully.
         Current function value: 0.323565
         Iterations 8

1st reduced model AIC: 30.927
2nd reduced model AIC: 29.297


Even better! Let's see if we have any insignificant variables still:

In [47]:
print(reduced_model2_fit.summary())

                           Logit Regression Results                           
Dep. Variable:                  Group   No. Observations:                   36
Model:                          Logit   Df Residuals:                       33
Method:                           MLE   Df Model:                            2
Date:                Tue, 26 Mar 2024   Pseudo R-squ.:                  0.5290
Time:                        19:35:24   Log-Likelihood:                -11.648
converged:                       True   LL-Null:                       -24.731
Covariance Type:            nonrobust   LLR p-value:                 2.082e-06
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
X1             5.5291      2.841      1.946      0.052      -0.039      11.097
X3             2.7476      1.025      2.681      0.007       0.739       4.756
Intercept     -5.1018      1.945     -2.623      0.0

The $X_1$ confidence interval still contains zero, so that means it might not be significant. Let's see how the AIC is when we remove it:

In [50]:
x_train4 = x_train[['X3', 'Intercept']]
reduced_model3 = sm.Logit(y_train, x_train4)
reduced_model3_fit = reduced_model3.fit()

print("\n2st reduced model AIC: {:.3f}".format(reduced_model2_aic))

reduced_model3_aic = aic(llf=reduced_model3.loglike(reduced_model3_fit.params), 
                        nobs=x_train4.shape[0], 
                        df_modelwc=len(reduced_model3_fit.params))
print("3nd reduced model AIC: {:.3f}".format(reduced_model3_aic))

Optimization terminated successfully.
         Current function value: 0.393315
         Iterations 7

2st reduced model AIC: 29.297
3nd reduced model AIC: 32.319


So removing it increases the AIC, meaning that the model is better off with $X_1$ after all. This gives us our final model:

In [53]:
logit_model = reduced_model2
logit_model_fit = reduced_model2_fit
print(logit_model_fit.summary())

                           Logit Regression Results                           
Dep. Variable:                  Group   No. Observations:                   36
Model:                          Logit   Df Residuals:                       33
Method:                           MLE   Df Model:                            2
Date:                Tue, 26 Mar 2024   Pseudo R-squ.:                  0.5290
Time:                        19:42:46   Log-Likelihood:                -11.648
converged:                       True   LL-Null:                       -24.731
Covariance Type:            nonrobust   LLR p-value:                 2.082e-06
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
X1             5.5291      2.841      1.946      0.052      -0.039      11.097
X3             2.7476      1.025      2.681      0.007       0.739       4.756
Intercept     -5.1018      1.945     -2.623      0.0

## Coefficient estimation
Since the coefficients are already given in the fitted model, we simply need to exponentiate them to get $\exp(\beta_i)$ for $i \in [0,2]$, giving the following:

In [67]:
print("beta", "{:>18}".format("exp(beta_i)"),end=None)
print(logit_model_fit.params.apply(math.exp))

beta        exp(beta_i)
X1           251.905737
X3            15.605700
Intercept      0.006086
dtype: float64


The 95% confidence intervals for these exponentiated coefficients are easily derived as well:

In [74]:
ci_df = logit_model_fit.conf_int(0.95)
ci_df[0] = ci_df[0].apply(math.exp)
ci_df[1] = ci_df[1].apply(math.exp)
print(ci_df)

                    0           1
X1         210.803092  301.022627
X3          14.634346   16.641527
Intercept    0.005387    0.006875


These are multiplicative confidence intervals, which means that they are the <i>product</i> of the estimate and $\exp(\text{MoE})$ (margin of error) or the estimate and $\exp(-\text{MoE})$ for the low end of the interval. 

## Error analysis

## Descision boundary

# Machine Learning Approach

# Model Comparison

# Discussion